### Calculate stemness index (mRNAsi)

In [ ]:
# Loading Packages
library(Seurat)
#library(ReactomePA)
# library(org.Hs.eg.db)
library(ggplot2)
library(ggrepel)
library(ggpubr)
library(readxl)
library(survival)
library(survminer)
library(Publish)

setwd("/media/bio/Disk/Research Data/EBV/omicverse")
options(future.globals.maxSize = 100 * 1024^3)  # 10 GB

options(Seurat.object.assay.version = "v3")

Sys.setenv(LANGUAGE = "en")

In [ ]:
# library(sceasy)
# use_condaenv('npc_env')
               
# sceasy::convertFormat('Processed Data/scRNA_Epi_CNV_Traj.h5ad', from="anndata", to="seurat",
#                       outFile='Processed Data/scRNA_Epi_CNV_Traj.rds')

In [ ]:
scRNA_Epi_CNV_Traj <- readRDS('Processed Data/scRNA_Epi_CNV_Traj.rds')
colnames(scRNA_Epi_CNV_Traj@meta.data)

In [ ]:
scRNA_Epi_CNV_Traj_exp=GetAssayData(subset(scRNA_Epi_CNV_Traj),slot='counts')
scRNA_Epi_CNV_Traj_exp[1:5, 1:5]

In [ ]:
predict.mRNAsi <- function(exp, modelPath='model.rda') {
  load(modelPath)
  common <- intersect(names(model.RNA$w), rownames(exp))
  X <- exp[common, ]
  w <- model.RNA$w[common]
  
  mRNAsi <- apply(X, 2, function(z) {cor(z, w, method="sp", use="complete.obs")})
  mRNAsi <- mRNAsi - min(mRNAsi)
  mRNAsi <- mRNAsi / max(mRNAsi)
}

In [ ]:
scRNA_Epi_mRNAsi <- predict.mRNAsi(scRNA_Epi_CNV_Traj_exp, 'Dataset/mRNAsi/model.rda')
head(scRNA_Epi_mRNAsi)

In [ ]:
scRNA_Epi_mRNAsi <- data.frame(scRNA_Epi_mRNAsi)
scRNA_Epi_mRNAsi$CellID <- rownames(scRNA_Epi_mRNAsi)

df <- scRNA_Epi_CNV_Traj@meta.data
df$CellID <- rownames(df)

df_scRNA_Epi_mRNAsi <-  merge(x = scRNA_Epi_mRNAsi, y = df, by = "CellID", all = TRUE)
head(df_scRNA_Epi_mRNAsi)

In [ ]:
df_scRNA_Epi_mRNAsi$Epi_celltype <- factor(df_scRNA_Epi_mRNAsi$Epi_celltype, levels = c('Normal ECs', 'EBV- ECs', 'EBV+ ECs', 'EBV- CCs', 'EBV+ CCs'))

In [ ]:
library(compareGroups)
table <- compareGroups(Epi_celltype~scRNA_Epi_mRNAsi+pt_via+CytoTRACE2_Score, data=df_scRNA_Epi_mRNAsi, method=4)
export_table <- createTable(table,digits.p = 3, show.all=T)
export_table

In [ ]:
ggviolin1 <- ggviolin(df_scRNA_Epi_mRNAsi,
                     x = "Epi_celltype", 
                     y = "cnv_score",
                     xlab = "",
                     ylab = "CNV Score",
                     color = "Epi_celltype", 
                     palette = "jama",
                     add = c("boxplot"), 
                     shape = "Epi_celltype" 
) + 
  stat_compare_means(comparisons = list(c('EBV- ECs','Normal ECs'), c('EBV+ ECs','Normal ECs'), c('EBV- CCs','EBV- ECs'), c('EBV+ CCs','EBV+ ECs')), 
                     method = "wilcox.test", label = "p.format", label.y = c(0.1,0.15,0.2,0.25)) +
    theme(legend.position = "none", 
        axis.text.x = element_text(angle = 45, hjust = 1)) 

ggviolin1

ggsave("Results/06.Epithelial/CNV_score_Epi_celltype.pdf", ggviolin1, width = 6, height = 5, units = c("in"), dpi = 300, limitsize = TRUE)

In [ ]:
ggviolin1 <- ggviolin(df_scRNA_Epi_mRNAsi,
                     x = "Epi_celltype", 
                     y = "scRNA_Epi_mRNAsi",
                     xlab = "",
                     ylab = "mRNAsi",
                     color = "Epi_celltype", 
                     palette = "jama",
                     add = c("boxplot"), 
                     shape = "Epi_celltype" 
) + 
  stat_compare_means(comparisons = list(c('EBV- ECs','Normal ECs'),c('EBV+ ECs','Normal ECs'), c('EBV- CCs','Normal ECs'),  
  c('EBV+ CCs','Normal ECs'), c('EBV+ ECs','EBV- ECs'), c('EBV+ CCs','EBV- CCs')), 
                     method = "wilcox.test", label = "p.format", label.y = c(0.5,0.6,0.7,0.8,0.9,1.0)) +
    theme(legend.position = "none", 
        axis.text.x = element_text(angle = 45, hjust = 1))

ggviolin1

ggsave("Results/06.Epithelial/mRNAsi_Epi_celltype.pdf", ggviolin1, width = 6, height = 5, units = c("in"), dpi = 300, limitsize = TRUE)


In [ ]:
ggviolin2 <- ggviolin(df_scRNA_Epi_mRNAsi,
                     x = "Epi_celltype", 
                     y = "CytoTRACE2_Score",
                     xlab = "",
                     ylab = "CytoTRACE2 Score",
                     color = "Epi_celltype", 
                     palette = "jama",
                     add = c("boxplot"),
                     shape = "Epi_celltype" 
) + stat_compare_means(comparisons = list(c('EBV- ECs','Normal ECs'),c('EBV+ ECs','Normal ECs'), c('EBV- CCs','Normal ECs'),  
  c('EBV+ CCs','Normal ECs'), c('EBV+ ECs','EBV- ECs'), c('EBV+ CCs','EBV- CCs')), 
                     method = "wilcox.test", label = "p.format", label.y = c(0.45,0.5,0.55,0.6,0.65,0.7)) +
    theme(legend.position = "none", 
        axis.text.x = element_text(angle = 45, hjust = 1)) 

ggviolin2

ggsave("Results/06.Epithelial/CytoTRACE2_Score_Epi_celltype.pdf", ggviolin2, width = 6, height = 5, units = c("in"), dpi = 300, limitsize = TRUE)

In [ ]:
ggviolin3 <- ggviolin(df_scRNA_Epi_mRNAsi,
                     x = "Epi_celltype", 
                     y = "pt_via",
                     xlab = "",
                     ylab = "StaVia Pseudotime",
                     color = "Epi_celltype", 
                     palette = "jama",
                     add = c("boxplot"),
                     shape = "Epi_celltype"
) + stat_compare_means(comparisons = list(c('EBV- ECs','Normal ECs'),c('EBV+ ECs','Normal ECs'), c('EBV- CCs','Normal ECs'),  
  c('EBV+ CCs','Normal ECs'), c('EBV+ ECs','EBV- ECs'), c('EBV+ CCs','EBV- CCs')), 
                     method = "wilcox.test", label = "p.format", label.y = c(0.2,0.4,0.6,0.8,1.0,1.2)) +
    theme(legend.position = "none",
        axis.text.x = element_text(angle = 0, hjust = 1))

ggviolin3

ggsave("Results/06.Epithelial/pt_via_Epi_celltype.pdf", ggviolin3, width =8 , height = 3, units = c("in"), dpi = 300, limitsize = TRUE)

In [ ]:
write.csv(df_scRNA_Epi_mRNAsi, "Processed Data/df_scRNA_Epi_CNV_Traj_mRNAsi.csv", row.names=FALSE)

In [ ]:
library(ggplot2)
vocanol_Epi_data <- read_excel("Results/06.Epithelial/DEG_results.xlsx", sheet = 1)
head(vocanol_Epi_data)

In [ ]:
vocanol_Epi_data = subset(vocanol_Epi_data, ...1!='AC090498.1')
vocanol_Epi_data = subset(vocanol_Epi_data, padj!='NA')
head(vocanol_Epi_data)

In [ ]:
vocanol_Epi_data %>% arrange(padj) %>% head(30)

In [ ]:
library(ggrepel)
library(dplyr)

ggplot(vocanol_Epi_data, aes(log2FoldChange, -log10(padj))) +
  geom_hline(yintercept = -log10(0.05), linetype = "dashed", color = "#999999") +
  geom_vline(xintercept = c(-1.2, 1.2), linetype = "dashed", color = "#999999") +
  geom_point(aes(size = -log10(padj), color = -log10(padj))) +
  scale_color_gradientn(values = seq(0, 1, 0.2), 
                        colors = c("#39489f", "#39bbec", "#f9ed36", "#f38466", "#b81f25")) +
  scale_size_continuous(range = c(1, 3)) +
  scale_x_continuous(limits = c(-5.5, 10.5), breaks = seq(-5, 10, by = 2.5)) + 
  theme_bw() +
  theme(panel.grid = element_blank(),
        legend.position = c(0.7, 0.9),
        legend.justification = c(0, 1),
        axis.title.x = element_text(size = 16),
        axis.title.y = element_text(size = 16), 
        axis.text.x = element_text(size = 14), 
        axis.text.y = element_text(size = 14)
  ) +
  guides(col = guide_colourbar(title = "-Log10 q-value"), size = "none") +
  geom_text_repel(data = vocanol_Epi_data %>% arrange(padj) %>% head(30),
                  aes(label = ...1, color = -log10(padj)), 
                  size = 3, max.overlaps = 50) +
  xlab("Log2FC") +
  ylab("-Log10(q-value)")

ggsave("Results/06.Epithelial/06.Epithelial_DEG_VacanoPlot.pdf", height = 4, width = 5)